# Transfer learning with PyTorch
We're going to train a neural network to classify dogs and cats.

## Init, helpers, utils, ...

In [1]:
from pprint import pprint
import random
import datetime
import time

from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np

#from ppt import utils
#from ppt.utils import attr

%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torchvision
from torchvision.datasets.folder import ImageFolder, default_loader

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
# Training helpers
def get_trainable(model_params):
    return (p for p in model_params if p.requires_grad)


def get_frozen(model_params):
    return (p for p in model_params if not p.requires_grad)


def all_trainable(model_params):
    return all(p.requires_grad for p in model_params)


def all_frozen(model_params):
    return all(not p.requires_grad for p in model_params)


def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False


# list(get_trainable(model.parameters()))
# list(get_frozen(model.parameters()))
# all_trainable(model.parameters())
# all_frozen(model.parameters())

# The Data - DogsCatsDataset

## Transforms

In [6]:
from torchvision import transforms

IMG_SIZE = 224  #224  #defined by NN model input
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_trans = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),  #256  #(IMG_SIZE, IMG_SIZE)  # some images are pretty small
    #transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_trans = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),  #256  #(IMG_SIZE, IMG_SIZE)
    #transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

## Dataset

In [7]:
# not necessary
#from ppt.utils import DogsCatsDataset

In [8]:
# sample data set
#train_ds = DogsCatsDataset("../data/raw", "sample/train", transform=train_trans)
#val_ds = DogsCatsDataset("../data/raw", "sample/valid", transform=val_trans)
#BATCH_SIZE = 2

# full data set
# use ppt.utils
#train_ds = DogsCatsDataset("../data/raw", "train", transform=train_trans)
#val_ds = DogsCatsDataset("../data/raw", "valid", transform=val_trans)
# use pytorch_version default
train_ds = ImageFolder("../data/raw/DUI/train", transform=train_trans, loader=default_loader)
val_ds = ImageFolder("../data/raw/DUI/valid", transform=train_trans, loader=default_loader)

BATCH_SIZE = 128  #2  #256  #512  #32  #220 for resnet152 on Dell Presison 5520 laptop, 400 for resnet18

n_classes = 2

In [9]:
len(train_ds), len(val_ds)

(13787, 1421)

## DataLoader
Batch loading for datasets with multi-processing and different sample strategies.

In [10]:
from torch.utils.data import DataLoader


train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)
val_dl = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

# The Model
PyTorch offers quite a few [pre-trained networks](https://pytorch.org/docs/stable/torchvision/models.html) for you to use:
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3

And there are more available via [pretrained-models.pytorch](https://github.com/Cadene/pretrained-models.pytorch)
- NASNet,
- ResNeXt,
- InceptionV4,
- InceptionResnetV2, 
- Xception, 
- DPN,
- ...

In [11]:
from torchvision import models

model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True)
#model = models.resnet101(pretrained=True)
#model = models.resnet152(pretrained=True)

In [ ]:
model

In [13]:
for param in model.parameters():
    param.requires_grad = True

#for param in model.parameters():
    #pprint(param)

In [14]:
'''# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False'''

'# Freeze all parameters\nfor param in model.parameters():\n    param.requires_grad = False'

In [15]:
'''freeze_all(model.parameters())
assert all_frozen(model.parameters())'''

'freeze_all(model.parameters())\nassert all_frozen(model.parameters())'

Replace the last layer with a linear layer. New layers have `requires_grad = True`.

In [16]:
'''model.fc = nn.Linear(512, n_classes)  # according to the model, 512 for resnet18, 2048 for resnet50 & resnet101 & resnet152'''

'model.fc = nn.Linear(512, n_classes)  # according to the model, 512 for resnet18, 2048 for resnet50 & resnet101 & resnet152'

In [17]:
'''all_frozen(model.parameters())'''

'all_frozen(model.parameters())'

In [18]:
'''# repetitive
def get_model(n_classes=2):
    model = models.resnet18(pretrained=True)
    freeze_all(model.parameters())
    model.fc = nn.Linear(512, n_classes)
    return model'''

#model = get_model().to(device)

model = model.to(device)

# The Loss

In [19]:
criterion = nn.CrossEntropyLoss()

# The Optimizer

In [20]:
optimizer = torch.optim.Adam(
    get_trainable(model.parameters()),
    # model.fc.parameters(),
    lr=0.001,
    # momentum=0.9,
)

# The Train Loop

In [21]:
N_EPOCHS = 10  #1  #2  #10

In [ ]:
for epoch in range(N_EPOCHS):
    
    # start epoch
    start_time = time.time()
    start_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    print(f'  Start Time: {start_datetime}')
    
    # Train
    model.train()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    for X, y in train_dl:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        # with torch.set_grad_enabled(True):
        y_ = model(X)
        loss = criterion(y_, y)

        loss.backward()
        optimizer.step()
        
        # Statistics
        print(f"    batch loss: {loss.item():0.3f}")
        _, y_label_ = torch.max(y_, 1)
        correct += (y_label_ == y).sum().item()
        running_loss += loss.item() * X.shape[0]
    
    print(f"  Train Loss: {running_loss / len(train_dl.dataset)}")
    print(f"  Train Acc:  {correct / len(train_dl.dataset)}")
    
    
    # Eval
    model.eval()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    with torch.no_grad():  # IMPORTANT
        for X, y in val_dl:
            X, y = X.to(device), y.to(device)
                    
            y_ = model(X)
        
            _, y_label_ = torch.max(y_, 1)
            correct += (y_label_ == y).sum().item()
            
            loss = criterion(y_, y)
            running_loss += loss.item() * X.shape[0]
    
    print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
    print(f"  Valid Acc:  {correct / len(val_dl.dataset)}")
    
    # end epoch
    end_time = time.time()
    end_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    time_elapsed = end_time - start_time
    datetime_elapsed = str(datetime.timedelta(seconds = time_elapsed))
    print(f'  End Time: {end_datetime}')
    print(f'  Time Elapsed: {datetime_elapsed}')
    print()

** -- above is the result of crop extent data set -- **

# Predict with Trained Model

Pickle the trained model and predict image with it. 

In [44]:
# save the trained model weights
model_weights_path = '../data/saved_model_weights/resnet18_whole'

# !!!only use it when you WANT to save a trained model weights!!!
#torch.save(model.state_dict(), model_weights_path)

In [45]:
# load the trained model
from torchvision import models

model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True)
#model = models.resnet101(pretrained=True)
#model = models.resnet152(pretrained=True)

model.load_state_dict(torch.load(model_weights_path))

model = model.to(device)

In [ ]:
# use trained model to predict a pair of people in a new image

# test data set
test_ds = ImageFolder("../data/raw/DUI/test", transform=val_trans, loader=default_loader)
print(f'len(test_ds) = {len(test_ds)}. ')

test_dl = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
)

print(f'test_ds[99]: \n{test_ds[99]}')
print(f'test_ds[99][1]: \n{test_ds[99][1]}')

Recommended approach for saving a model
There are two main approaches for serializing and restoring a model.

### The first (recommended) saves and loads only the model parameters:

torch.save(the_model.state_dict(), PATH)

Then later:

the_model = TheModelClass(*args, **kwargs)
the_model.load_state_dict(torch.load(PATH))

### The second saves and loads the entire model:

torch.save(the_model, PATH)
Then later:

the_model = torch.load(PATH)
However in this case, the serialized data is bound to the specific classes and the exact directory structure used, so it can break in various ways when used in other projects, or after some serious refactors.

In [ ]:
# predict

# Eval
model.eval()  # IMPORTANT

with torch.no_grad():  # IMPORTANT
    for X, y in test_dl:
        X, y = X.to(device), y.to(device)
        print(f'y: \t\t\t{y}')

        y_ = model(X)
        _, y_label_ = torch.max(y_, 1)
        print(f'y_label_: \t\t{y_label_}')
        
        is_correct = 'correct' if y_label_ == y else 'wrong'
        print(f'is_correct: \t{is_correct}\n')

#print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
#print(f"  Valid Acc:  {correct / len(val_dl.dataset)}")